In [1]:
my_input = """Player 1:
14
29
25
17
13
50
33
32
7
37
26
34
46
24
3
28
18
20
11
1
21
8
44
10
22

Player 2:
5
38
27
15
45
40
43
30
35
9
48
12
16
47
42
4
2
31
41
39
23
19
36
49
6
"""

test_input="""Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10"""

In [2]:
from collections import deque
class Combat:
    def __init__(self, hand1, hand2):
        self.rounds = 0
        self.player1 = deque(hand1)
        self.player2 = deque(hand2)
        
    def play1(self):
        card1 = self.player1.popleft()
        card2 = self.player2.popleft()
        
        if card1 > card2:
            winner = self.player1
            card_order = (card1, card2)
        else:
            winner = self.player2
            card_order = (card2, card1)
            
        winner.append(card_order[0])
        winner.append(card_order[1])
        
        self.rounds += 1
        
    def play_to_end(self):
        while (len(self.player1) > 0) and (len(self.player2) > 0):
            self.play1()
            
    def score(self):
        if len(self.player1) > 0:
            winner = self.player1
        else:
            winner = self.player2
            
        total = 0
        for i in range(len(winner)):
            multiplier = i+1
            index = -(i+1)
            total += winner[index] * multiplier
        return total
            
            
            

In [3]:
data = {}
for player_id, player in enumerate(my_input.split("\n\n")):
    lines = player.splitlines()
    player_id = int(lines[0].split()[1][:-1])
    cards = [ int(card) for card in lines[1:]]
    data[player_id] = cards
combat = Combat(data[1], data[2])
combat.play_to_end()

In [4]:
combat.rounds

1015

In [5]:
combat.score()

34324

# Problem 2

Recursive Combat still starts by splitting the cards into two decks. Then, the game consists of a series of rounds with a few changes:

 - Before either player deals a card, if there was a previous round in this game that had exactly the same cards in the same order in the same players' decks, the game instantly ends in a win for player 1. Previous rounds from other games are not considered. (This prevents infinite games of Recursive Combat, which everyone agrees is a bad idea.)
 - Otherwise, this round's cards must be in a new configuration; the players begin the round by each drawing the top card of their deck as normal.
 - If both players have at least as many cards remaining in their deck as the value of the card they just drew, the winner of the round is determined by playing a new game of Recursive Combat (see below).
 - Otherwise, at least one player must not have enough cards left in their deck to recurse; the winner of the round is the player with the higher-value card.

As in regular Combat, the winner of the round (even if they won the round by winning a sub-game) takes the two cards dealt at the beginning of the round and places them on the bottom of their own deck (again so that the winner's card is above the other card). Note that the winner's card might be the lower-valued of the two cards if they won the round due to winning a sub-game. If collecting cards by winning the round causes a player to have all of the cards, they win, and the game ends.

In [6]:
class CursedCombat:
    def __init__(self, hand1, hand2):
        self.rounds = 0
        self.player1 = deque(hand1)
        self.player2 = deque(hand2)
        self.cursedConfigs = set()
        self.gameOver = False
        
    def checkCurse(self):
        config = (tuple(self.player1), tuple(self.player2))
        if self.gameOver or (config in self.cursedConfigs):
            self.gameOver = True
            return True # we are cursed
        self.cursedConfigs.add(config)
        return False
        
    def play1(self):
        if not self.checkCurse():
            card1 = self.player1.popleft()
            card2 = self.player2.popleft()
            
            if ((card1 <= len(self.player1)) 
            and (card2 <= len(self.player2))):
                recursed_combat = CursedCombat(list(self.player1)[:card1], 
                                               list(self.player2)[:card2]).play_to_end()
                if recursed_combat.winner() == 1:
                    winner = self.player1
                    card_order = (card1, card2)
                else:
                    winner = self.player2
                    card_order = (card2, card1)
                
            else:
                if card1 > card2:
                    winner = self.player1
                    card_order = (card1, card2)
                else:
                    winner = self.player2
                    card_order = (card2, card1)
            
            winner.append(card_order[0])
            winner.append(card_order[1])
        
            self.rounds += 1
            
        
    def play_to_end(self):
        while ((not self.gameOver) 
               and (len(self.player1) > 0) 
               and (len(self.player2) > 0)):
            self.play1()
        return self
            
    def score(self):
        if self.gameOver or (len(self.player1) > 0):
            winner = self.player1
        else:
            winner = self.player2
            
        total = 0
        for i in range(len(winner)):
            multiplier = i+1
            index = -(i+1)
            total += winner[index] * multiplier
        return total
    
    def winner(self):
        if self.gameOver or (len(self.player1) > 0):
            return 1
        else:
            return 2
            
            
            

In [7]:
data = {}
for player_id, player in enumerate(my_input.split("\n\n")):
    lines = player.splitlines()
    player_id = int(lines[0].split()[1][:-1])
    cards = [ int(card) for card in lines[1:]]
    data[player_id] = cards
combat = CursedCombat(data[1], data[2])
combat.play_to_end()

In [8]:
combat.rounds

347

In [9]:
combat.score()

33259